Link tham khảo: XGboot
https://github.com/dataquestio/project-walkthroughs/blob/master/bitcoin_price/prediction.ipynb  (https://www.youtube.com/watch?v=TF2Nx_ifmrU&t=2550s)


- This data is time series it's sequential -> we don't use Cross-Validation or any of the model ML techniques to evaluate error. 
- TimeSeriesSplit, which is a specific type of cross-validation technique used for time series data. It's important to use time-series cross-validation when dealing with sequential data to avoid training on future data.


In [61]:
# Load Data
import pandas as pd
# data = pd.read_csv("https://raw.githubusercontent.com/lavibula/ML20222.PredictionBitcoin/main/data/data.csv")
data = pd.read_csv(r"C:\Users\Administrator\OneDrive - Hanoi University of Science and Technology\ITE10 - Data Science and AI - HUST\20222\ML\Source_Codes\ML20222.PredictionBitcoin\data\data.csv")
# print(data.info())
data = data.sort_values('Date', ascending=True).reset_index(drop=True) # drop=True bỏ 'index' col cũ. 
data = data.drop('BTC_change', axis=1)
data

,Date,BTC_close,BTC_open,BTC_high,BTC_low,difficulty,addresses_active_count,sum_lock_weight,mean_lock_size_ytes,total_fees_usd,...,ETH,LTC,XRP,DOGE,COPPER,GOLD,SILVER,SPX,JP225,DJI
0,2016-03-10,415.8,412.8,417.5,410.3,158427203767,445273,426654988,6.881532e+05,1.731272e+04,...,0.00,0.00,0.00000,0.000000,0.0000,0.00,0.000,0.00,0.0,0.0
1,2016-03-11,419.1,415.8,422.4,415.1,158427203767,434658,398582424,6.227850e+05,1.710193e+04,...,0.00,0.00,0.00000,0.000000,0.0000,0.00,0.000,0.00,0.0,0.0
2,2016-03-12,410.4,419.1,420.7,407.0,158427203767,374730,331208848,5.750154e+05,1.398444e+04,...,0.00,0.00,0.00000,0.000000,0.0000,0.00,0.000,0.00,0.0,0.0
3,2016-03-13,412.4,410.4,415.9,409.6,158427203767,421585,334817852,6.293569e+05,1.460678e+04,...,0.00,0.00,0.00000,0.000000,0.0000,0.00,0.000,0.00,0.0,0.0
4,2016-03-14,414.3,412.4,416.1,411.2,158427203767,451902,437739524,7.199663e+05,1.689298e+04,...,0.00,0.00,0.00000,0.000000,0.0000,0.00,0.000,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2656,2023-06-18,26339.7,26515.0,26679.3,26290.6,52350439455487,883864,603064705,1.985675e+06,7.946708e+05,...,1720.98,77.20,0.48699,0.062107,3.8738,1969.45,24.198,0.00,0.0,0.0
2657,2023-06-19,26845.9,26339.7,27029.7,26295.1,52350439455487,920552,567091224,1.758290e+06,8.514319e+05,...,1737.06,77.51,0.49341,0.062429,3.8643,1964.05,24.062,0.00,0.0,0.0
2658,2023-06-20,28307.7,26845.9,28393.0,26665.5,52350439455487,951926,543133484,1.677592e+06,1.052364e+06,...,1791.61,80.31,0.49270,0.063108,3.8835,1947.70,23.234,4388.71,33155.0,34356.0
2659,2023-06-21,29996.9,28307.7,30769.5,28270.5,52350439455487,920555,483181576,1.611773e+06,1.347976e+06,...,1889.87,85.12,0.49995,0.065644,3.9135,1944.90,22.810,4365.69,33390.0,34235.0


In [62]:
wiki = pd.read_csv("wiki_dataquestio_edited.csv")
# print(wiki.info())
data = pd.merge(data, wiki, on='Date', how='inner') #quan sát info 2 df, 
                       #how=inner: ghép chéo-lấy các hàng chung trong cả 2 df #outer
data.set_index('Date', inplace=True) #inplace=True; thay đổi được áp dụng trực tiếp vào df gốc
# print(data.info())

In [63]:
data["BTC_tomorrow"] = data["BTC_close"].shift(-1)
data["BTC_target"] = (data["BTC_tomorrow"] > data["BTC_close"]).astype(int)
data["BTC_target"].value_counts()  # BTC_target: BTC_direction, compare BTC_close with BTC_tomorrow.
                                    # return 1 if nay < mai. 
print(data)

            BTC_close  BTC_open  BTC_high  BTC_low      difficulty   
Date                                                                 
2016-03-10      415.8     412.8     417.5    410.3    158427203767  \
2016-03-11      419.1     415.8     422.4    415.1    158427203767   
2016-03-12      410.4     419.1     420.7    407.0    158427203767   
2016-03-13      412.4     410.4     415.9    409.6    158427203767   
2016-03-14      414.3     412.4     416.1    411.2    158427203767   
...               ...       ...       ...      ...             ...   
2022-09-04    19999.9   19832.2   20017.8  19594.7  30977051760460   
2022-09-05    19793.1   19999.9   20042.9  19650.7  30977051760460   
2022-09-06    18786.4   19793.4   20169.3  18723.3  30977051760460   
2022-09-07    19281.5   18786.3   19445.9  18548.4  30977051760460   
2022-09-08    19317.4   19281.5   19444.7  19034.5  30977051760460   

            addresses_active_count  sum_lock_weight  mean_lock_size_ytes   
Date         

In [64]:
data

,BTC_close,BTC_open,BTC_high,BTC_low,difficulty,addresses_active_count,sum_lock_weight,mean_lock_size_ytes,total_fees_usd,mean_hash_rate,...,GOLD,SILVER,SPX,JP225,DJI,edit_count,sentiment,neg_sentiment,BTC_tomorrow,BTC_target
Date,,,,,,,,,,,,,,,,,,,,,
2016-03-10,415.8,412.8,417.5,410.3,158427203767,445273,426654988,6.881532e+05,17312.723752,1.220696e+06,...,0.0,0.000,0.00,0.0,0.0,2.166667,-0.325637,0.425815,419.1,1
2016-03-11,419.1,415.8,422.4,415.1,158427203767,434658,398582424,6.227850e+05,17101.930760,1.260073e+06,...,0.0,0.000,0.00,0.0,0.0,2.166667,-0.293752,0.409148,410.4,0
2016-03-12,410.4,419.1,420.7,407.0,158427203767,374730,331208848,5.750154e+05,13984.439830,1.134066e+06,...,0.0,0.000,0.00,0.0,0.0,2.166667,-0.293752,0.409148,412.4,1
2016-03-13,412.4,410.4,415.9,409.6,158427203767,421585,334817852,6.293569e+05,14606.780891,1.047436e+06,...,0.0,0.000,0.00,0.0,0.0,2.300000,-0.296397,0.425815,414.3,1
2016-03-14,414.3,412.4,416.1,411.2,158427203767,451902,437739524,7.199663e+05,16892.976657,1.197070e+06,...,0.0,0.000,0.00,0.0,0.0,2.366667,-0.328309,0.459148,415.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-04,19999.9,19832.2,20017.8,19594.7,30977051760460,774639,387954834,8.655431e+05,209607.207443,2.586994e+08,...,0.0,0.000,0.00,0.0,0.0,0.400000,-0.108480,0.150000,19793.1,0
2022-09-05,19793.1,19999.9,20042.9,19650.7,30977051760460,942780,475262812,1.184311e+06,281853.665763,2.356013e+08,...,1712.9,17.908,0.00,0.0,0.0,0.400000,-0.108480,0.150000,18786.4,0
2022-09-06,18786.4,19793.4,20169.3,18723.3,30977051760460,927513,466361631,1.261380e+06,289585.926070,2.186626e+08,...,0.0,0.000,3908.19,27625.0,31166.0,0.433333,-0.081502,0.150000,19281.5,1


In [65]:
print(data.columns)

Index(['BTC_close', 'BTC_open', 'BTC_high', 'BTC_low', 'difficulty',
       'addresses_active_count', 'sum_lock_weight', 'mean_lock_size_ytes',
       'total_fees_usd', 'mean_hash_rate', 'xfer_cnt', 'mean_tx_size_usd',
       'ETH', 'LTC', 'XRP', 'DOGE', 'COPPER', 'GOLD', 'SILVER', 'SPX', 'JP225',
       'DJI', 'edit_count', 'sentiment', 'neg_sentiment', 'BTC_tomorrow',
       'BTC_target'],
      dtype='object')


In [66]:
# Training our baseline ML model: RandomForest
from sklearn.ensemble import RandomForestClassifier   #Ensemble Method
from sklearn.metrics import precision_score # ValueError: continuous is not supported, nhãn 0/1

rf_model = RandomForestClassifier(n_estimators=100, min_samples_split=50, random_state=1)
                               # số cây, individual decision tree should not split its nodes unless it has at least 50 samples. 

train = data.iloc[:-250]
test = data.iloc[-250:]

predictors = ['BTC_close', 'BTC_open', 'BTC_high', 'BTC_low', 'difficulty',
       'addresses_active_count', 'sum_lock_weight', 'mean_lock_size_ytes',
       'total_fees_usd', 'mean_hash_rate', 'xfer_cnt', 'mean_tx_size_usd',
       'ETH', 'LTC', 'XRP', 'DOGE', 'COPPER', 'GOLD', 'SILVER', 'SPX', 'JP225',
       'DJI', 'edit_count', 'sentiment', 'neg_sentiment']  # drop BTC_tomorrow and BTC_target
rf_model.fit(train[predictors], train["BTC_target"])

y_pred_test = rf_model.predict(test[predictors])
precision_score(test["BTC_target"], y_pred_test)

0.4714285714285714

In [67]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(random_state=1, learning_rate=.1, n_estimators=150)
xgb_model.fit(train[predictors], train["BTC_target"])

y_pred_test = xgb_model.predict(test[predictors])
precision_score(test["BTC_target"], y_pred_test)

0.47953216374269003

Output: 0.45454545454545453

In [68]:
# Đóng gói backtest function: 
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["BTC_target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="predictions")
    combined = pd.concat([test["BTC_target"], preds], axis=1)
    return combined

# Evaluating error with backtesting
def backtest(data, model, predictors, start=1095, step=150): # 150 - every 6 months
    """
    backtest: train-test Time Series
    """
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [ ]:
# Đóng gói def Evaluate_Model and def Plot_Model
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

def Evaluate_Model(y_test, y_pred_test):
  """
  parameters: `y_test`, `y_pred_test`
  calculates evaluation metrics,
    including R-squared, MSE, RMSE, MAE, MAPE,
      and directional accuracy (DA) 
  """
  print('Coefficient of determination R2:', r2_score(y_test, y_pred_test)) # print(model.score(X_test, y_test))
  print('Mean Squared Error MSE:', mean_squared_error(y_test, y_pred_test))
  print('Root Mean Squared Error RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_test)))
  print('Mean Absolute Error MAE:', mean_absolute_error(y_test, y_pred_test))
  MAPE = mean_absolute_percentage_error(y_test, y_pred_test)
  # MAPE = np.mean(np.abs((np.array(y_test) - np.array(y_pred_test)) / y_test))
  print('Mean Absolute Percentage Error MAPE%', f"{round(MAPE*100, 2)}%")

  # Calculate the directional movements
  y_pred_test_direction = np.sign(np.diff(y_pred_test))  # np.sign chuyển 1/-1
  y_test_direction = np.sign(np.diff(y_test))
  compare = (y_pred_test_direction == y_test_direction) # compare 
  DA = accuracy_score(compare, np.ones_like(compare)) #[True True ... True]
  print("Directional Accurancy DA%:", f"{round(DA*100, 2)}%")
    
def Plot_Model(y_test, y_pred_test, num_samples):
  """
  Scatter Plot(Biểu đồ phân tán) with x_axis: y_test, y_axis: y_pred_test
  Print test_pred_df
  Line Plot with x_axis: position point, y_axis: Value and 2 line: y_test, y_pred_test 
  """
  # Gộp 2 Plot scatterplot and lineplot
  plt.subplots(figsize=(15,7)) # return tuple (Figure, Axes). obj Figure: toàn bộ hình ảnh, obj Axes: các vùng trực quan. 
  ax1 = plt.subplot(121)
  ax2 = plt.subplot(122)

  # Scatter plot
  ax1.scatter(y_test, y_pred_test)
  ax1.set_title("Predictions vs Actual")  #plt.title
  ax1.set_xlabel("y_true")                #plt.xlabel
  ax1.set_ylabel("y_pred_test")           #plt.ylabel

  x = range(int(max(y_test))) #vẽ đường chéo giữa.
                              # type y_test from df: <class 'pandas.core.series.Series'>, tương tự df, but have only 1 col 
  ax1.plot(x, 'r') # Opt speed, we can use x = np.arange(int(max(y_test))).astype(int)

  # Line plot with x_axis: position point, y_axis: Value and 2 line: y_test, y_pred_test 
  #1 plot vẽ tập Test nhưng show ra x 6000 điểm -> vì plot ra index của hàng ban đầu. 
  #2 Muốn plot ra tập sample với ít điểm giá trị hơn. 
  #3 Muốn plot hiện điểm 1, 2, 3, ... chứ không phải điểm 0.0, 0.5, 1.0, ...
  test_pred_df = pd.DataFrame({'y_test': y_test, 'y_pred_test': y_pred_test}) # Create df 3 col: index, y_test, y_pred_test 
  print(test_pred_df)

  test_pred_subset_df = pd.DataFrame({'y_test_few': y_test[:num_samples], 'y_pred_test_few': y_pred_test[:num_samples]}) #2
  test_pred_subset_df = test_pred_subset_df.reset_index(drop=True) #1 Add a reset index and Remove old index
  
  # sns.lineplot(data=test_pred_subset_df) # data = df
  sns.lineplot(data=test_pred_subset_df).set(xticks=range(0, num_samples+1, int(num_samples/5)+1)) 
                                        # set x_axis ticks (dấu kiểm) hiện ít điểm hơn trên x
  ax2.set_title("Test Predictions")
  ax2.set_xlabel("Point") # tên hàng 
  ax2.set_ylabel("Value")
  plt.show() # show những plots above

# y_test = [10 , 15, 20, 25, 30, 35, 40]
# y_pred_test = [-10, -15, -20, -25, -30, -35, -40]
# Evaluate_Model(y_test, y_pred_test)
# Plot_Model(y_test, y_pred_test, 3)

In [70]:
from xgboost import XGBClassifier, XGBRegressor

xgb_model = XGBRegressor(random_state=1, learning_rate=.1, n_estimators=150)
xgb_model.fit(train[predictors], train["BTC_target"])

y_pred_test = xgb_model.predict(test[predictors])
Evaluate_Model(test["BTC_target"], y_pred_test)

Coefficient of determination R2: -0.0916289098218317
Mean Squared Error MSE: 0.271160621199743
Root Mean Squared Error RMSE: 0.5207308529362774
Mean Absolute Error MAE: 0.4955498734116554
Mean Absolute Percentage Error MAPE% 1.2916337070393262e+17%
Directional Accurancy DA%: 27.31%


In [71]:
#Using backtest with RF XGB

# RF
rf_model = RandomForestClassifier(n_estimators=100, min_samples_split=50, random_state=1)
                               # số cây, individual decision tree should not split its nodes unless it has at least 50 samples. 
predictions = backtest(data, rf_model, predictors)
precision_score(predictions["BTC_target"], predictions["predictions"]) # tính độ chính xác của model sau các lần backtesting
print(precision_score(predictions["BTC_target"], predictions["predictions"])) # tính độ chính xác của model sau các lần backtesting


# Using an XGBoost model
from xgboost import XGBClassifier # XGBoost, LightGBM, AdaBoost, CatBoost-Boosting-Ensemble Methods.  
xgb_model = XGBClassifier(random_state=1, learning_rate=.1, n_estimators=50)
predictions = backtest(data, xgb_model, predictors)
precision_score(predictions["BTC_target"], predictions["predictions"]) # tính độ chính xác của model sau các lần backtesting

0.5159010600706714


0.5051395007342144

## Update model: 



This data is time series it's sequential -> we don't use Cross-Validation or any of the model ML techniques to evaluate error. 
Có thể chạy một vòng lặp để kiểm tra các giá trị tham số khác nhau và xem xét độ chính xác của model để chọn các giá trị tối ưu cho các tham số này. Tuy nhiên, hãy nhớ rằng dữ liệu Time-Series là dữ liệu tuần tự theo thời gian, do đó không sử dụng Cross-Validation hay các kỹ thuật ML techniques khác để đánh giá sai số.
- `learning_rate`: 
- `n_estimators`: 
- `max_depth`:
- `subsample`:
- `colsample_bytree`:
- `gamma`: . 
- `min_child_weight`: 

Grid Search hoặc Randomized Search để tìm các giá trị tối ưu cho các tham số. Ngoài ra, có thể sử dụng các kỹ thuật khác như Feature Selection, Ensemble Learning,... để cải thiện hiệu suất của mô hình XGBoost trên dữ liệu Time-Series.

Bạn đúng là đối với bài toán time series, chúng ta cần sử dụng một phương pháp cross-validation khác để đảm bảo rằng các tập train và test không trộn lẫn giữa quá khứ và tương lai. Đối với bài toán time series, chúng ta thường sử dụng phương pháp Time Series Cross Validation.

Trong Time Series Cross Validation, thay vì chia dữ liệu thành các fold ngẫu nhiên, chúng ta sẽ giữ lại các điểm dữ liệu gần nhất cho training set và sử dụng các điểm dữ liệu xa hơn cho validation hoặc test set. Một số phương pháp Time Series Cross Validation phổ biến bao gồm: 

1. **Walk-Forward Validation**: Chúng ta bắt đầu với một window size ban đầu và cố gắng dự đoán các giá trị trong window tiếp theo. Sau đó, chúng ta tăng window size và tiếp tục quá trình này cho đến khi tiêu chuẩn dừng được đạt được.

2. **Expanding Window**: Chúng ta bắt đầu với một window size ban đầu và sau đó mở rộng cửa sổ bằng cách thêm dữ liệu mới vào phía sau cửa sổ. Quá trình này được lặp lại cho đến khi chúng ta sử dụng hết dữ liệu.

3. **Rolling Window**: Chúng ta di chuyển cửa sổ đi qua dữ liệu và sử dụng các giá trị trong cửa sổ đó để dự đoán giá trị tiếp theo. Cửa sổ được di chuyển bằng một bước nhất định và quá trình này được lặp lại cho đến khi chúng ta sử dụng hết dữ liệu.

Với phương pháp Time Series Cross Validation, chúng ta có thể đảm bảo rằng các dữ liệu ở tương lai không sẽ bị trộn lẫn vào tập train và test ở quá khứ.

Chúng ta có thể lấy một ví dụ đơn giản như sau: Giả sử chúng ta có tập dữ liệu về doanh thu hàng tháng của một công ty từ năm 2016 đến năm 2020. Ta muốn sử dụng mô hình XGBoost để dự đoán doanh thu cho các tháng tiếp theo trong tương lai.

Với cách chia dữ liệu thông thường, chúng ta có thể chia ngẫu nhiên các tháng thành tập huấn luyện và tập kiểm tra. Tuy nhiên, điều này không phù hợp trong bài toán chuỗi thời gian vì dữ liệu trong tương lai sẽ được dựa trên các giá trị trong quá khứ.

Thay vào đó, chúng ta có thể sử dụng TimeSeriesSplit để chia dữ liệu thành các fold, sao cho các fold có dữ liệu từ quá khứ trước đó sẽ được sử dụng làm tập huấn luyện, và dữ liệu chỉ nhận giá trị trong tương lai sẽ được sử dụng làm tập kiểm tra. Ví dụ, nếu chúng ta chia dữ liệu thành 3 fold, mỗi fold sẽ bao gồm:

- Fold 1: Dữ liệu từ tháng 1/2016 đến tháng 12/2017 (tập huấn luyện) và dữ liệu từ tháng 1/2018 đến tháng 12/2018 (tập kiểm tra).
- Fold 2: Dữ liệu từ tháng 1/2016 đến tháng 12/2018 (tập huấn luyện) và dữ liệu từ tháng 1/2019 đến tháng 12/2019 (tập kiểm tra).
- Fold 3: Dữ liệu từ tháng 1/2016 đến tháng 12/2019 (tập huấn luyện) và dữ liệu từ tháng 1/2020 đến tháng 12/2020 (tập kiểm tra).

Một khi chúng ta đã có các fold như trên, chúng ta có thể sử dụng mô hình XGBoost để huấn luyện trên tập huấn luyện của từng fold, sau đó sử dụng tập kiểm tra của từng fold để đánh giá hiệu suất của mô hình.

Việc sử dụng TimeSeriesSplit sẽ giúp đảm bảo rằng việc huấn luyện và đánh giá mô hình được thực hiện trên các dữ liệu phù hợp với yêu cầu của bài toán chuỗi thời gian.

In [72]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from xgboost import XGBClassifier
import numpy as np

predictors = ['BTC_close', 'BTC_open', 'BTC_high', 'BTC_low', 'BTC_volume',
       'Active_Addr_Cnt', 'Difficulty', 'Mean_Block_Size(in_bytes)',
       'Sum_Block_Weight', 'Total_Fees(USD)', 'Mean_Hashrate', 'Xfer_Cnt',
       'Mean_Tx_size(native_units)', 'ETH', 'LTC', 'DOGE', 'XRP', 'GOLD',
       'SILVER', 'COPPER', 'S&P500', 'DJI', 'JP225', "edit_count", "sentiment", "neg_sentiment"]

xgb = XGBClassifier() # XGBoost

# Lựa chọn giá trị siêu tham số: dựa vào data và value thông thường XGBoost
param_grid = {   
    'learning_rate': [0.1, 0.3, 0.5],
    'n_estimators': [50, 100, 150], 
    'max_depth': [3,5,7], 
#      'colsample_bytree': [0.7, 0.8, 0.9],
#       'subsample': [0.7, 0.8, 0.9],
#       'gamma'
#       'min_child_samples': [1, 5, 10]
}

# thay đổi cv sang TimeSeriesSplit-specific tyoe of cv tech used for time series- 
tscv = TimeSeriesSplit(n_splits=3)
grid_search_model = GridSearchCV(estimator=xgb,
                           param_grid=param_grid,
                           cv=tscv.split(train[predictors])) # sử dụng TimeSeriesSplit

# fit và tìm siêu tham số tối ưu
grid_search_model.fit(train[predictors], train["BTC_target"])
print("Best parameters found: ", grid_search_model.best_params_)

best_params_xgb_model = grid_search_model.best_estimator_
print(best_params_xgb_model)


KeyError: "['BTC_volume', 'Active_Addr_Cnt', 'Difficulty', 'Mean_Block_Size(in_bytes)', 'Sum_Block_Weight', 'Total_Fees(USD)', 'Mean_Hashrate', 'Xfer_Cnt', 'Mean_Tx_size(native_units)', 'S&P500'] not in index"

Best parameters found:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}

In [ ]:

from xgboost import XGBClassifier

xgb_model = XGBClassifier(random_state=1, learning_rate=0.1, max_depth=3, n_estimators=50)
xgb_model.fit(train[predictors], train["BTC_target"])

y_pred_train = xgb_model.predict(train[predictors])
precision_score(train["BTC_target"], y_pred_train) #bình thường đứng 1 mình là print rồi, nhưng khi lồng vào các đoạn thì lại mất thêm câu print
print(precision_score(train["BTC_target"], y_pred_train)) #nhưng khi lồng vào các đoạn thì lại mất thêm câu print

y_pred_test = xgb_model.predict(test[predictors])
precision_score(test["BTC_target"], y_pred_test)
print(precision_score(test["BTC_target"], y_pred_test))
Evaluate_Model(test["BTC_target"], y_pred_test)

In [ ]:
# Use the best parameters found by GridSearchCV to create a new XGBoost model

# Use the best parameters found by GridSearchCV to create a new XGBoost model
best_xgb_model = XGBRegressor(random_state=1, learning_rate=grid_search_model.best_params_['learning_rate'],
                              n_estimators=grid_search_model.best_params_['n_estimators'],
                              max_depth=grid_search_model.best_params_['max_depth'])

# Train the model on the entire training dataset with the best hyperparameters
best_xgb_model.fit(train[predictors], train["BTC_target"])
y_pred_test = best_xgb_model.predict(test[predictors])

Evaluate_Model(test["BTC_target"], y_pred_test)


In [ ]:

# Using an XGBoost model
from xgboost import XGBClassifier # XGBoost, LightGBM, AdaBoost, CatBoost-Boosting-Ensemble Methods.  
best_xgb_model = XGBClassifier(random_state=1, learning_rate=grid_search_model.best_params_['learning_rate'],
                              n_estimators=grid_search_model.best_params_['n_estimators'],
                              max_depth=grid_search_model.best_params_['max_depth'])

predictions = backtest(data, best_xgb_model, predictors)
precision_score(predictions["BTC_target"], predictions["predictions"]) # tính độ chính xác của model sau các lần backtesting

In [ ]:
from matplotlib import pyplot as plt


features = train[predictors].columns
importance_values = best_params_xgb_model.feature_importances_

plt.barh(y=range(len(features)), width=importance_values, tick_label=features)
plt.show()